# 1. Useful functions

This notebook contains useful functions that can be imported and reused within a Jupyter notebook.

In [1]:
import dns
from dns import resolver
import requests
from urllib.parse import urlparse
import geoip2.database
import pyspark.sql.types as t

# Get A-Record from domain
def getARecords(domain):
    try:
        result = dns.resolver.resolve(domain, 'A')
        return list(map(lambda ipval: ipval.to_text(), result))
    except dns.resolver.NXDOMAIN:
        return None
    except dns.resolver.NoAnswer:
        return None
    except dns.resolver.NoNameservers:
        return None
    except dns.resolver.Timeout:
        return None

# Get A-Record error from domain
def getARecords_error(domain):
    try:
        result = dns.resolver.resolve(domain, 'A')
        return 0
    except dns.resolver.NXDOMAIN:
        return 1
    except dns.resolver.NoAnswer:
        return 2
    except dns.resolver.NoNameservers:
        return 3
    except dns.resolver.Timeout:
        return 4

# Get IPv6-Record from domain
def IPv6Record(domain):
    try:
        result = dns.resolver.resolve(domain, 'AAAA')
        return True
    except dns.resolver.NXDOMAIN:
        return None
    except dns.resolver.NoAnswer:
        return False
    except dns.resolver.NoNameservers:
        return None
    except dns.resolver.Timeout:
        return None
    
def IPv6Record_error(domain):
    try:
        result = dns.resolver.resolve(domain, 'AAAA')
        return 0
    except dns.resolver.NXDOMAIN:
        return 1
    except dns.resolver.NoAnswer:
        return 2
    except dns.resolver.NoNameservers:
        return 3
    except dns.resolver.Timeout:
        return 4

# Get MX-Record from domain   
def getMXRecords(domain):
    try:
        result = dns.resolver.resolve(domain, 'MX')
        return list(map(lambda mail: mail.exchange.to_text(), result))
    except dns.resolver.NXDOMAIN:
        return None
    except dns.resolver.NoAnswer:
        return None
    except dns.resolver.NoNameservers:
        return None
    except dns.exception.Timeout:
        return None

# Get MX-Record error from domain  
def getMXRecords_error(domain):
    try:
        result = dns.resolver.resolve(domain, 'MX')
        return 0
    except dns.resolver.NXDOMAIN:
        return 1
    except dns.resolver.NoAnswer:
        return 2
    except dns.resolver.NoNameservers:
        return 3
    except dns.exception.Timeout:
        return 4
    
# Get Redirect from domain
def getRedirectUrl(domain):
    try:
        response = requests.get("http://" + domain, timeout=5)
        url = urlparse(response.url).netloc
        url = ('.'.join(url.split('.')[-2:]))
        return url
    except requests.exceptions.ConnectionError:
        return None
    except requests.exceptions.ReadTimeout:
        return None
    except requests.exceptions.TooManyRedirects:
        return None

# Get status code from domain
def getStatusCodeUrl(domain):
    try:
        response = requests.get("http://" + domain, timeout=5)
        return response.status_code
    except requests.exceptions.ConnectionError:
        return None
    except requests.exceptions.ReadTimeout:
        return None
    except requests.exceptions.TooManyRedirects:
        return None

# Get SOA information from domain
def getSOAInformation(domain):
    try:
        result = dns.resolver.resolve(domain, 'SOA')
        return list(map(lambda soa: soa.to_text(), result))
    except dns.resolver.NXDOMAIN:
        return None
    except dns.resolver.NoAnswer:
        return None
    except dns.resolver.NoNameservers:
        return None
    except dns.exception.Timeout:
        return None
    
    
# Get SOA information error from domain  
def getSOAInformation_error(domain):
    try:
        result = dns.resolver.resolve(domain, 'SOA')
        return 0
    except dns.resolver.NXDOMAIN:
        return 1
    except dns.resolver.NoAnswer:
        return 2
    except dns.resolver.NoNameservers:
        return 3
    except dns.exception.Timeout:
        return 4
    
    
# Get nameserver from domain    
def getNameServers(domain):
    try:
        result = dns.resolver.resolve(domain, 'NS')
        return list(map(lambda soa: soa.to_text(), result))
    except dns.resolver.NXDOMAIN:
        return None
    except dns.resolver.NoAnswer:
        return None
    except dns.resolver.NoNameservers:
        return None
    except dns.exception.Timeout:
        return None
    
    
# Get nameserver error from domain  
def getNameServers_error(domain):
    try:
        result = dns.resolver.resolve(domain, 'NS')
        return 0
    except dns.resolver.NXDOMAIN:
        return 1
    except dns.resolver.NoAnswer:
        return 2
    except dns.resolver.NoNameservers:
        return 3
    except dns.exception.Timeout:
        return 4
    
# Get geolite2 location data
def getGeoLite2_Location(ip):
    with geoip2.database.Reader('../data/GeoLite2-City.mmdb') as reader:
        response = reader.city(ip)
        iso_code = response.country.iso_code
        country = response.country.name
        city = response.city.name
        postal = response.postal.code
        latitude = response.location.latitude
        longitude = response.location.longitude
        return t.Row('iso_code', 'country', 'city', 'postal', 'latitude', 'longitude')(iso_code, country, city, postal, latitude, longitude)

# Get geolite2 ASN data
def getGeoLite2_ASN(ip):
    with geoip2.database.Reader('../data/GeoLite2-ASN.mmdb') as reader:
        response = reader.asn(ip)
        asn = response.autonomous_system_number
        asno = response.autonomous_system_organization
        return t.Row('autonomous_system_number', 'autonomous_system_organization')(asn, asno)

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def execute_threaded_fn(fn, args, log_at):
    futures, results = [], []

    with ThreadPoolExecutor(max_workers=1024) as executor:
        i, f, s = 0, 0, 0
        for arg in args:
            i = i + 1
            if i % log_at == 0: print("futures:", i)
            futures.append(executor.submit(fn, arg))

        for future in futures:
            try:
                result = future.result(timeout=60)
                results.append(result)
                s = s + 1
            except Exception:
                results.append(None)
                f = f + 1
            finally:
                if (s+f) % log_at == 0: print(f"[Processed: {s + f}] Successful: {s} | Failed: {f}")
    return results